<a href="https://colab.research.google.com/github/Ash-Stat/An-Si-Hyun/blob/data/kaggle_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA

In [92]:
import pandas as pd
data=pd.read_csv('./train.csv')
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [93]:
# find a NaN
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [94]:
# drop featureless columns
data=data.drop(['PassengerId','Name'],axis=1)
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,211536,13.0000,NaN,S
887,1,1,female,19.0,0,0,112053,30.0000,B42,S
888,0,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,1,male,26.0,0,0,111369,30.0000,C148,C


In [95]:
# find unique values
# 잘못 입력된 값이 있는지 확인
for i in data.columns:
  print(i,':', data[i].unique())

Survived : [0 1]
Pclass : [3 1 2]
Sex : ['male' 'female']
Age : [22.   38.   26.   35.     nan 54.    2.   27.   14.    4.   58.   20.
 39.   55.   31.   34.   15.   28.    8.   19.   40.   66.   42.   21.
 18.    3.    7.   49.   29.   65.   28.5   5.   11.   45.   17.   32.
 16.   25.    0.83 30.   33.   23.   24.   46.   59.   71.   37.   47.
 14.5  70.5  32.5  12.    9.   36.5  51.   55.5  40.5  44.    1.   61.
 56.   50.   36.   45.5  20.5  62.   41.   52.   63.   23.5   0.92 43.
 60.   10.   64.   13.   48.    0.75 53.   57.   80.   70.   24.5   6.
  0.67 30.5   0.42 34.5  74.  ]
SibSp : [1 0 3 4 2 5 8]
Parch : [0 1 2 5 3 4 6]
Ticket : ['A/5 21171' 'PC 17599' 'STON/O2. 3101282' '113803' '373450' '330877'
 '17463' '349909' '347742' '237736' 'PP 9549' '113783' 'A/5. 2151'
 '347082' '350406' '248706' '382652' '244373' '345763' '2649' '239865'
 '248698' '330923' '113788' '347077' '2631' '19950' '330959' '349216'
 'PC 17601' 'PC 17569' '335677' 'C.A. 24579' 'PC 17604' '113789' '2677'


In [96]:
# 칼럼별 생존과의 연관성을 확인 : 오즈비
# 데이터셋 준비
import numpy as np
odds=pd.crosstab(index=data.Pclass, columns=data.Survived,values=data.Survived, aggfunc=pd.value_counts )
odds.loc[1]=odds.loc[1]/216
odds.loc[2]=odds.loc[2]/184
odds.loc[3]=odds.loc[3]/491
odds=odds[[1,0]]
odds

Survived,1,0
Pclass,,
1,0.629630,0.370370
2,0.472826,0.527174
3,0.242363,0.757637


In [97]:
#Pclass=1 vs 2
print('odds of Pclass 1 vs 2 :', 1/((80*87)/(136*97)))
print('odds of Pclass 2 vs 3 :', 1/((97*119)/(87*372)))
print('odds of Pclass 1 vs 3 :', 1/((80*119)/(136*372)))

odds of Pclass 1 vs 2 : 1.8954022988505748
odds of Pclass 2 vs 3 : 2.803777180975483
odds of Pclass 1 vs 3 : 5.314285714285714


# Pclass2가 1보다 생존비율이 높다
# Pclass3이 2보다 생존비율이 높다.
# Pclass3이 1보다 생존비율이 월등히 높다.

In [98]:
# 아노바 분석
chi_data=pd.crosstab(index=data.Pclass, columns=data.Survived,values=data.Survived, aggfunc=pd.value_counts )
chi_data

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


In [99]:
import numpy as np
from scipy.stats import chi2_contingency
chi, p, dof, expected = chi2_contingency(chi_data)

print('Survived and Pclass')
print()
print(f"chi 스퀘어 값: {chi}",
      f"p-value (0.05): {p}",
      f"자유도 수: {dof}",
      f"기대값: \n{pd.DataFrame(expected)}",
      f"측정값: \n{chi_data}", sep = "\n" )
print('p-value가 0.05 이하 이므로, 귀무가설을 기각한다. 즉, 생존과 Pclass는 관계가 있다.')

Survived and Pclass

chi 스퀘어 값: 102.88898875696056
p-value (0.05): 4.549251711298793e-23
자유도 수: 2
기대값: 
            0           1
0  133.090909   82.909091
1  113.373737   70.626263
2  302.535354  188.464646
측정값: 
Survived    0    1
Pclass            
1          80  136
2          97   87
3         372  119
p-value가 0.05 이하 이므로, 귀무가설을 기각한다. 즉, 생존과 Pclass는 관계가 있다.


In [100]:
#성별과 생존
odds_sex_survived=pd.crosstab(index=data.Sex, columns=data.Survived,values=data.Survived, aggfunc=pd.value_counts )
odds_sex_survived=odds_sex_survived[[1,0]]
odds_sex_survived.loc['female']=odds_sex_survived.loc['female']/314
odds_sex_survived.loc['male']=odds_sex_survived.loc['male']/577
odds_sex_survived

Survived,1,0
Sex,,
female,0.742038,0.257962
male,0.188908,0.811092


In [101]:
print('odds of Sex female vs male :', (odds_sex_survived[1][0] * odds_sex_survived[0][1]) / (odds_sex_survived[1][1] * odds_sex_survived[0][0]))

odds of Sex female vs male : 12.3506625891947


# 여성이 남성보다 생존확률이 월등히 높다

In [102]:
# 아노바 분석
chi_Sex=pd.crosstab(index=data.Sex, columns=data.Survived,values=data.Survived, aggfunc=pd.value_counts )
chi_Sex=chi_Sex[[1,0]]
chi1, p1, dof1, expected1 = chi2_contingency(chi_Sex)

print('Survived and Sex')
print()
print(f"chi 스퀘어 값: {chi1}",
      f"p-value (0.05): {p1}",
      f"자유도 수: {dof1}",
      f"기대값: \n{pd.DataFrame(expected1)}",
      f"측정값: \n{chi_Sex}", sep = "\n" )
print('p-value가 0.05 이하 이므로, 귀무가설을 기각한다. 즉, 생존과 Sex는 관계가 있다.')

Survived and Sex

chi 스퀘어 값: 260.71702016732104
p-value (0.05): 1.1973570627755645e-58
자유도 수: 1
기대값: 
            0           1
0  120.525253  193.474747
1  221.474747  355.525253
측정값: 
Survived    1    0
Sex               
female    233   81
male      109  468
p-value가 0.05 이하 이므로, 귀무가설을 기각한다. 즉, 생존과 Sex는 관계가 있다.


# Age의 결측값을 어떻게 채울까??

Maximum Likelihood & 베이즈 공식 이용

In [103]:
sample=data[['Survived','Sex','Pclass','Age']]
sample=sample.drop_duplicates()
sample

,Survived,Sex,Pclass,Age
0,0,male,3,22.0
1,1,female,1,38.0
2,1,female,3,26.0
3,1,female,1,35.0
4,0,male,3,35.0
...,...,...,...,...
869,1,male,3,4.0
871,1,female,1,47.0
872,0,male,1,33.0
879,1,female,1,56.0


In [104]:
#Age를 나이대로 바꾼다.

for older,ind in zip(sample['Age'],sample.index):
  if older <10 :
    sample['Age'].loc[ind]=0 
  elif older <20:
    sample['Age'].loc[ind]=1
  elif older <30:
    sample['Age'].loc[ind]=2
  elif older <40:
    sample['Age'].loc[ind]=3
  elif older <50:
    sample['Age'].loc[ind]=4
  elif older <60:
    sample['Age'].loc[ind]=5
  elif older <70:
    sample['Age'].loc[ind]=6
  else:
    sample['Age'].loc[ind]=7
sample

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Survived,Sex,Pclass,Age
0,0,male,3,2.0
1,1,female,1,3.0
2,1,female,3,2.0
3,1,female,1,3.0
4,0,male,3,3.0
...,...,...,...,...
869,1,male,3,0.0
871,1,female,1,4.0
872,0,male,1,3.0
879,1,female,1,5.0


In [105]:
# 일단 결측값을 채우기 전에 완전히 주어진 데이터로만 오즈비와 카이제곱 검정을 해본다.

#성별과 생존
odds_age_survived=pd.crosstab(index=sample.Age, columns=sample.Survived,values=sample.Survived, aggfunc=pd.value_counts )
odds_age_survived=odds_age_survived[[1,0]]
odds_age_survived
odds_age_survived['sum']=odds_age_survived[1]+odds_age_survived[0]
odds_age_survived[1]=odds_age_survived[1]/odds_age_survived['sum']
odds_age_survived[0]=odds_age_survived[0]/odds_age_survived['sum']
odds_age_survived=odds_age_survived.drop(['sum'],axis=1)
odds_age_survived

Survived,1,0
Age,,
0.0,0.684211,0.315789
1.0,0.555556,0.444444
2.0,0.453333,0.546667
3.0,0.493151,0.506849
4.0,0.407407,0.592593
5.0,0.424242,0.575758
6.0,0.400000,0.600000
7.0,0.437500,0.562500


In [106]:
def odds(dat,init,m):
  for n in range(init,m):
    print('odds %ds vs %ds:' % (init,n+1),(dat[1].iloc[init] * dat[0].iloc[n+1]) / (dat[1].iloc[n+1] * dat[0].iloc[init]))

In [107]:
odds(odds_age_survived,0,7)
odds(odds_age_survived,1,7)
odds(odds_age_survived,2,7)
odds(odds_age_survived,3,7)
odds(odds_age_survived,4,7)
odds(odds_age_survived,5,7)
odds(odds_age_survived,6,7)

odds 0s vs 1s: 1.7333333333333334
odds 0s vs 2s: 2.6127450980392157
odds 0s vs 3s: 2.226851851851852
odds 0s vs 4s: 3.1515151515151514
odds 0s vs 5s: 2.9404761904761907
odds 0s vs 6s: 3.25
odds 0s vs 7s: 2.785714285714286
odds 1s vs 2s: 1.5073529411764708
odds 1s vs 3s: 1.2847222222222225
odds 1s vs 4s: 1.8181818181818183
odds 1s vs 5s: 1.6964285714285716
odds 1s vs 6s: 1.8749999999999998
odds 1s vs 7s: 1.6071428571428574
odds 2s vs 3s: 0.8523035230352305
odds 2s vs 4s: 1.2062084257206207
odds 2s vs 5s: 1.1254355400696865
odds 2s vs 6s: 1.24390243902439
odds 2s vs 7s: 1.0662020905923346
odds 3s vs 4s: 1.4152334152334152
odds 3s vs 5s: 1.3204633204633205
odds 3s vs 6s: 1.459459459459459
odds 3s vs 7s: 1.250965250965251
odds 4s vs 5s: 0.9330357142857145
odds 4s vs 6s: 1.0312499999999998
odds 4s vs 7s: 0.8839285714285714
odds 5s vs 6s: 1.1052631578947365
odds 5s vs 7s: 0.9473684210526316
odds 6s vs 7s: 0.8571428571428571


# 10대 이전의 생존확률은 그 어느 연령층보다 높다.

# 10대의 생존확률은 10대 이전의 생존확률을 제외하고 나머지 연련층에 비해 모두 높다.

# 20대의 생존확률은 30대를 제외하고 나머지 연령층보다 미세하게 높다.

# 30대의 생존확률은 10대와 10대 이전의 연령층 제외하고 나머지 연령층에 비해 모두 높다.

In [109]:
# 아노바 분석
chi_Age=pd.crosstab(index=sample.Age, columns=sample.Survived,values=sample.Survived, aggfunc=pd.value_counts )
chi_Age=chi_Age[[1,0]]
chi2, p2, dof2, expected2 = chi2_contingency(chi_Age)

print('Survived and Age')
print()
print(f"chi 스퀘어 값: {chi2}",
      f"p-value (0.05): {p2}",
      f"자유도 수: {dof2}",
      f"기대값: \n{pd.DataFrame(expected2)}",
      f"측정값: \n{chi_Age}", sep = "\n" )
print('p-value가 0.05 초과 이므로, 귀무가설을 기각할 수 없다. 즉, 생존과 Age는 관계가 없다.')

Survived and Age

chi 스퀘어 값: 9.614211809915117
p-value (0.05): 0.21150988639149484
자유도 수: 7
기대값: 
           0          1
0  18.510029  19.489971
1  21.919771  23.080229
2  36.532951  38.467049
3  35.558739  37.441261
4  26.303725  27.696275
5  16.074499  16.925501
6   7.306590   7.693410
7   7.793696   8.206304
측정값: 
Survived   1   0
Age             
0.0       26  12
1.0       25  20
2.0       34  41
3.0       36  37
4.0       22  32
5.0       14  19
6.0        6   9
7.0        7   9
p-value가 0.05 초과 이므로, 귀무가설을 기각할 수 없다. 즉, 생존과 Age는 관계가 없다.


# 오즈비의 결과와 카이제곱의 결과가 다른 현상을 발견했다. 둘중 뭐가 맞는 것 일까?